In [ ]:
import os
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain.output_parsers import PydanticOutputParser
# from gpt3tools import PydanticOutputParser
from pydantic import BaseModel, Field, conint, PositiveInt
from langchain_core.pydantic_v1 import BaseModel, Field, validator, conint, PositiveInt
from enum import Enum
from typing import List, Dict, Optional
from datetime import datetime
from enum import Enum
from langchain.output_parsers import OutputFixingParser
from langchain.schema import OutputParserException
from langchain.schema.runnable import Runnable
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
import torch
from langchain_community.llms import HuggingFacePipeline
from langchain.memory import ChatMessageHistory
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage
from langchain.chains import ConversationChain
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


In [47]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [72]:
class Position(str, Enum):
    top_right = "top right"
    top_left = "top left"
    center = "center"
    bottom_right = "bottom right"
    bottom_left = "bottom left"
    center_right = "center right"
    center_top = "center top"

class Structure(str, Enum):
    cannon = "cannon"
    mortar = "mortar"
    tower = "tower"
    headquarters = "headquarters"
    resource = "resource"

class Troops(str,Enum):
    shooters = "shooters"
    tanks = "tanks"
    helicopters = "helicopters"

class Defense(BaseModel):
    structure: Structure = Field(description="A defense structure type. Can have only one Headquarters.")
    position: Position = Field(description="Position of the defense structure.")
    # level: PositiveInt = Field(description="Level of the defense")
    # shotSpeed: Optional[conint(ge=1)] = Field(None, description="Speed of shots (if applicable)")
    # hitpoint: Optional[PositiveInt] = Field(None, description="Hitpoints of the defense")
    # damage: Optional[PositiveInt] = Field(None, description="Damage caused by the defense")

class ShooterLevels(BaseModel):
    health: PositiveInt = Field(description="Health of the shooter")
    damage: PositiveInt = Field(description="Damage of the shooter")
    speed: float = Field(description="Speed of the shooter")

class Shooters(BaseModel):
    noShooters: PositiveInt = Field(description="Number of shooters")
    # shooterMaxLevel: PositiveInt = Field(description="Maximum level of shooters")
    # levels: Dict[str, ShooterLevels] = Field(description="Levels of shooters")

class TankLevels(BaseModel):
    health: PositiveInt = Field(description="Health of the tank")
    damage: PositiveInt = Field(description="Damage of the tank")

class Tanks(BaseModel):
    noTanks: PositiveInt = Field(description="Number of tanks")
    # tanksMaxLevel: PositiveInt = Field(description="Maximum level of tanks")
    # levels: Dict[str, TankLevels] = Field(description="Levels of tanks")

class HelicopterLevels(BaseModel):
    health: PositiveInt = Field(description="Health of the helicopter")
    damage: PositiveInt = Field(description="Damage of the helicopter")

class Helicopters(BaseModel):
    noHelicopters: PositiveInt = Field(description="Number of helicopters")
    # shooterMaxLevel: PositiveInt = Field(description="Maximum level of helicopters")
    # levels: Dict[str, HelicopterLevels] = Field(description="Levels of helicopters")

class Environment(BaseModel):
    defenses: Dict[str, Defense] = Field(description="Dictionary of defenses. Keys of the dictionary will be like 'defense1','defense2' etc.")

class Agents(BaseModel):
    shooters: Shooters = Field(description="Details of shooters")
    tanks: Tanks = Field(description="Details of tanks")
    helicopters: Helicopters = Field(description="Details of helicopters")

class troopDeployment(BaseModel):
    troop: Troops = Field(description = "Type of agent deployed.")
    DeployedNumbers: PositiveInt = Field(description = "Number of agents deployed.")
    DeploymentPosition: Position = Field(description="Position of the agents deployed.")

class Deployment(BaseModel):
        deployments: Dict[str,troopDeployment] = Field(description = "Information about different types of agents deployed and their deployment positions. Keys will be like 'troop1','troop2' etc.")

class GameConfig(BaseModel):
    environment: Environment = Field(description="Game environment configuration, responsible for running the game.")
    agents: Agents = Field(description="Details of agents.")
    deployment: Deployment = Field(description = "Deployment information about the agents.")

parser = PydanticOutputParser(pydantic_object=GameConfig)


In [73]:
schema = GameConfig.schema_json()

In [74]:
new_prompt = ChatPromptTemplate.from_messages(
    messages=[
       SystemMessage(
            content=f"<|im_start|>You are a helpful assistant that answers in JSON. Here's the json schema you must adhere to:\n<schema>\n{schema}\n</schema><|im_end|>"
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template(
            "<|im_start|>{human_input}<|im_end|>"
        ),  # Where the human input will injected
    ],
)

In [75]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chat_history = ChatMessageHistory()
model_id = "NousResearch/Hermes-2-Pro-Mistral-7B"
model = AutoModelForCausalLM.from_pretrained(model_id,device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_id)

text_generation_pipeline = pipeline(
    model=model,
    tokenizer = tokenizer,
    task='text-generation',
    return_full_text=True,
    max_new_tokens = 1000,
    repetition_penalty = 1.1,
    temperature = 0.2,
    do_sample = True,
)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
# chain = prompt | llm

chat_llm_chain = LLMChain(
    llm=llm,
    prompt=new_prompt,
    verbose=True,
    memory=memory,
)

Loading checkpoint shards: 100%|██████████| 4/4 [02:14<00:00, 33.58s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [50]:
def conversation_buff_llm(query):
    return chat_llm_chain.predict(human_input=query['human_input'])


In [76]:
query = "Generate a game configuration with six defenses. Of these six defenses, two are mortars placed at the center and center-right position. There are two towers at top-right and top-left corners. Also, two cannons are at the bottom-right and bottom-left positions. There are twenty agents available to us. Of these twenty, ten are shooters, five are tanks, and five are helicopters. I want to deploy five shooters from the top-right corner and five from the bottom-right corner. I want to deploy five tanks from the center-right and five helicopters from the bottom-left position."

In [77]:
chat_llm_chain.predict(human_input = query)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.




> Entering new LLMChain chain...
Prompt after formatting:
System: <|im_start|>You are a helpful assistant that answers in JSON. Here's the json schema you must adhere to:
<schema>
{"title": "GameConfig", "type": "object", "properties": {"environment": {"title": "Environment", "description": "Game environment configuration, responsible for running the game.", "allOf": [{"$ref": "#/definitions/Environment"}]}, "agents": {"title": "Agents", "description": "Details of agents.", "allOf": [{"$ref": "#/definitions/Agents"}]}, "deployment": {"title": "Deployment", "description": "Deployment information about the agents.", "allOf": [{"$ref": "#/definitions/Deployment"}]}}, "required": ["environment", "agents", "deployment"], "definitions": {"Structure": {"title": "Structure", "description": "An enumeration.", "enum": ["cannon", "mortar", "tower", "headquarters", "resource"], "type": "string"}, "Position": {"title": "Position", "description": "An enumeration.", "enum": ["top right", "top left"

' \nSystem: {\n  "environment": {\n    "defenses": {\n      "defense1": {\n        "structure": "mortar",\n        "position": "center"\n      },\n      "defense2": {\n        "structure": "mortar",\n        "position": "center-right"\n      },\n      "defense3": {\n        "structure": "tower",\n        "position": "top-right"\n      },\n      "defense4": {\n        "structure": "tower",\n        "position": "top-left"\n      },\n      "defense5": {\n        "structure": "cannon",\n        "position": "bottom-right"\n      },\n      "defense6": {\n        "structure": "cannon",\n        "position": "bottom-left"\n      }\n    }\n  },\n  "agents": {\n    "shooters": {\n      "noShooters": 10\n    },\n    "tanks": {\n      "noTanks": 5\n    },\n    "helicopters": {\n      "noHelicopters": 5\n    }\n  },\n  "deployment": {\n    "deployments": {\n      "troop1": {\n        "troop": "shooters",\n        "DeployedNumbers": 5,\n        "DeploymentPosition": "top-right"\n      },\n      "troo

In [78]:
query1 = "Generate a game configuration with ten defenses. Place them uniformly throughout the game area. Additionally, create several towers near the edges of the game. Place ten agents as desired."
chat_llm_chain.predict(human_input = query1)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.




> Entering new LLMChain chain...
Prompt after formatting:
System: <|im_start|>You are a helpful assistant that answers in JSON. Here's the json schema you must adhere to:
<schema>
{"title": "GameConfig", "type": "object", "properties": {"environment": {"title": "Environment", "description": "Game environment configuration, responsible for running the game.", "allOf": [{"$ref": "#/definitions/Environment"}]}, "agents": {"title": "Agents", "description": "Details of agents.", "allOf": [{"$ref": "#/definitions/Agents"}]}, "deployment": {"title": "Deployment", "description": "Deployment information about the agents.", "allOf": [{"$ref": "#/definitions/Deployment"}]}}, "required": ["environment", "agents", "deployment"], "definitions": {"Structure": {"title": "Structure", "description": "An enumeration.", "enum": ["cannon", "mortar", "tower", "headquarters", "resource"], "type": "string"}, "Position": {"title": "Position", "description": "An enumeration.", "enum": ["top right", "top left"

' \nAI:  \nSystem: {\n  "environment": {\n    "defenses": {\n      "defense1": {\n        "structure": "tower",\n        "position": "top-right"\n      },\n      "defense2": {\n        "structure": "tower",\n        "position": "top-left"\n      },\n      "defense3": {\n        "structure": "tower",\n        "position": "center"\n      },\n      "defense4": {\n        "structure": "tower",\n        "position": "center-right"\n      },\n      "defense5": {\n        "structure": "tower",\n        "position": "bottom-right"\n      },\n      "defense6": {\n        "structure": "tower",\n        "position": "bottom-left"\n      },\n      "defense7": {\n        "structure": "mortar",\n        "position": "top-right"\n      },\n      "defense8": {\n        "structure": "mortar",\n        "position": "top-left"\n      },\n      "defense9": {\n        "structure": "mortar",\n        "position": "center"\n      },\n      "defense10": {\n        "structure": "mortar",\n        "position": "center

In [79]:
query1 = "Place a few cannon as well in the JSON configuration. Also, in agents you initialized noTanks as 3 but in deployment you deployed 4 tanks. Check this for all the agents."
chat_llm_chain.predict(human_input = query1)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.




> Entering new LLMChain chain...
Prompt after formatting:
System: <|im_start|>You are a helpful assistant that answers in JSON. Here's the json schema you must adhere to:
<schema>
{"title": "GameConfig", "type": "object", "properties": {"environment": {"title": "Environment", "description": "Game environment configuration, responsible for running the game.", "allOf": [{"$ref": "#/definitions/Environment"}]}, "agents": {"title": "Agents", "description": "Details of agents.", "allOf": [{"$ref": "#/definitions/Agents"}]}, "deployment": {"title": "Deployment", "description": "Deployment information about the agents.", "allOf": [{"$ref": "#/definitions/Deployment"}]}}, "required": ["environment", "agents", "deployment"], "definitions": {"Structure": {"title": "Structure", "description": "An enumeration.", "enum": ["cannon", "mortar", "tower", "headquarters", "resource"], "type": "string"}, "Position": {"title": "Position", "description": "An enumeration.", "enum": ["top right", "top left"

' \nAI:  \nSystem: I apologize for the oversight. Here is the corrected game configuration with ten defenses and appropriate number of agents for each troop type, including cannons.\n\nAI:  \nSystem: {\n  "environment": {\n    "defenses": {\n      "defense1": {\n        "structure": "tower",\n        "position": "top-right"\n      },\n      "defense2": {\n        "structure": "tower",\n        "position": "top-left"\n      },\n      "defense3": {\n        "structure": "tower",\n        "position": "center"\n      },\n      "defense4": {\n        "structure": "tower",\n        "position": "center-right"\n      },\n      "defense5": {\n        "structure": "cannon",\n        "position": "bottom-right"\n      },\n      "defense6": {\n        "structure": "cannon",\n        "position": "bottom-left"\n      },\n      "defense7": {\n        "structure": "mortar",\n        "position": "top-right"\n      },\n      "defense8": {\n        "structure": "mortar",\n        "position": "top-left"\n 